In [24]:
# Import dependencies
import pandas as pd
from sklearn import linear_model
from sqlalchemy import create_engine

In [25]:
#Import functions
def model(df, position, var):
    df = df.dropna()
    df = df[df['pos'] == position]   

    # the model will use different independent variables depending on position
    if position == 'QB':
        X = df[var]
    elif position == 'RB':
        X = df[var]
    elif position == 'WR' or 'TE':
        X = df[var]
    else:
        print('Invalid position entered')
        return

    y = df['next_yr_ppg']
    reg = linear_model.LinearRegression()
    reg.fit(X, y)
    return reg

def testModelAccuracy(model, df, position, var):
    # creates a df from the csvFile, drops na values and rows where FantPos
    # does not equal the position parameter
    df = df.dropna()
    df = df[df['pos'] == position]

    # the model will use different independent variables depending on position
    if position == 'QB':
        XTest = df[var]
    elif position == 'RB':
        XTest = df[var]
    elif position == 'WR' or 'TE':
        XTest = df[var]
    else:
        print('Invalid position entered')
        return

    yTest = df['next_yr_ppg']
    results = model.score(XTest, yTest)
    return results

def testModelDifference(model, df, position, var):
    # creates a df from the csvFile, drops na values and rows where FantPos
    # does not equal the position parameter
    df = df.dropna()
    df = df[df['pos'] == position]

    # the model will use different independent variables depending on position
    if position == 'QB':
        XTest = df[var]
    elif position == 'RB':
        XTest = df[var]
    elif position == 'WR' or 'TE':
        XTest = df[var]
    else:
        print('Invalid position entered')
        return


    yPred = model.predict(XTest)
    predAndActual = {'Name': df['player'], 'Predicted PPG': yPred,
                     'Actual PPG': df['next_yr_ppg']}

    # creates df from dictionary above
    database = pd.DataFrame(predAndActual)

    # creates a difference column which depicts the difference between the
    # predicted PPG and actual PPG
    database['Predicted PPG'] = database['Predicted PPG'].round(decimals=3)
    database['Difference'] = database['Predicted PPG'] - database['Actual PPG']
    database['Difference'] = database['Difference'].round(decimals=3)
    database['AbsDifference'] = database['Difference'].abs()
    meanDiff = round(database['Difference'].mean(), 3)
    medianDiff = round(database['Difference'].median(), 3)
    meanAbsDiff = round(database['AbsDifference'].mean(), 3)
    medianAbsDiff = round(database['AbsDifference'].median(), 3)

    return database, meanDiff, medianDiff, meanAbsDiff, medianAbsDiff

def testModel(model, test, train, position, var):
    accuracy = testModelAccuracy(model, train, position, var)
    differences = testModelDifference(model, test, position, var)
    meanDiff = differences[1]
    medDiff = differences[2]
    meanAbsDiff = differences[3]
    medAbsDiff = differences[4]

    print('The accuracy of the {0} model is {1}'.format(position, accuracy))
    print('The {0} model has an average error of {1} PPG and an average absolute error of {2} PPG'.format(position, meanDiff, meanAbsDiff))
    print('The {0} model has a median error of {1} PPG and a median absolute error of {2} PPG'.format(position, medDiff, medAbsDiff))
    print('\n')
    return

def useModel(model, df1, df2, position, var):
    df1.dropna()
    df2.dropna()
    # checks where the Fant Pos is the position given and returns a data frame
    # with only the rows that include said position
    df1 = df1[df1['pos'] == position]
    df2 = df2[df2['pos'] == position]
    df2 = df2[['plid','player','ppr_g']]
    newdf = df1.merge(df2,how='inner',left_on='plid',right_on='plid')

    # the model will use difference parameters based on position
    if position == 'QB':
        X = df1[var]
    elif position == 'RB':
        X = df1[var]
    elif position == 'WR' or 'TE':
        X = df1[var]
    else:
        print('Invalid position entered')
        return
    yPred = model.predict(X)

    # creates new df with the name of player, their position, and their
    # predicted PPG
    databaseDict = {'Name': df1['player'], 'Pos': df1['pos'], 'Predicted PPG': yPred}
    database = pd.DataFrame(databaseDict)
    database = database.merge(df2,how='inner',left_on='Name',right_on='player')
    database = database.drop(columns=['player','plid'])
    database['Difference'] = database['Predicted PPG'] - database['ppr_g']
    database = database.sort_values(by = ['Predicted PPG'], ascending = False)
    database['Predicted PPG'] = database['Predicted PPG'].round(decimals = 3)
    database['Predicted PPR'] = 17 * database['Predicted PPG']
    database['Predicted PPR'] = database['Predicted PPR'].round(decimals = 3)
    database['AbsDifference'] = database['Difference'].abs()
    meanDiff = round(database['Difference'].mean(), 3)
    medianDiff = round(database['Difference'].median(), 3)
    meanAbsDiff = round(database['AbsDifference'].mean(), 3)
    medianAbsDiff = round(database['AbsDifference'].median(), 3)
    print('The {0} model has an average error of {1} PPG and an average absolute error of {2} PPG'.format(position, meanDiff, meanAbsDiff))
    print('The {0} model has a median error of {1} PPG and a median absolute error of {2} PPG'.format(position, medianDiff, medianAbsDiff))


     # this adds a position rank column to the dataframe
#     posRank = []
#     posRankNum = 1
#     for index, row in newdf.iterrows():
#         posRank.append(posRankNum)
#         posRankNum += 1

#     database['PosRank'] = posRank


    database = database.reset_index(drop = True)

    return database

In [26]:
# Connect to Database
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'fballfinalproject.c6sg90iemyn2.us-east-2.rds.amazonaws.com' ## INSERT YOUR DB ADDRESS 
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres' ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'FFForesight5!!' ## CHANGE THIS TO YOUR POSTGRES PASSWORD 
POSTGRES_DBNAME = 'postgres' ## CHANGE THIS TO YOUR DATABASE NAME
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}').format(
    username=POSTGRES_USERNAME,
    password=POSTGRES_PASSWORD,
    ipaddress=POSTGRES_ADDRESS,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DBNAME)
# Create the connection
cnx = create_engine(postgres_str)

In [40]:
# Read in dataframes and merge
# fantasy = pd.read_sql_query('''SELECT * FROM fantasy;''', cnx)
# teams = pd.read_sql_query('''SELECT * FROM teams;''', cnx)
# advrush = pd.read_sql_query('''SELECT * FROM advrush;''', cnx)
# df = pd.merge(fantasy,teams[['tmid','passrate']],on='tmid', how='inner')
# df = pd.merge(df,advrush,on='plid',how='inner')
# df = df.drop(columns=['player_y', 'tm_y', 'pos_y',
#                       'age_y','G_y', 'gs_y','tmid_y'])
# df = df.rename(columns={'player_x':'player','tm_x':'tm','pos_x':'pos','age_x':'age','G_x':'G','gs_x':'gs','tmid_x':'tmid'})
# df
df = pd.read_sql_query('''SELECT fantasy.*,
                        teams.pasatt_g  AS tm_passatt, 
                        teams.passrate, teams.rushatt_g AS tm_rushatt,
                        ar.avgdot, ar.ybc_g, ar.ydsbc_rec, ar.yac_g, ar.ydsac_rec, ar.brktkl_g, ar.drop_g
                        FROM fantasy
                        INNER JOIN teams 
                        ON fantasy.tmid = teams.tmid
                        INNER JOIN advrec AS ar
                        ON fantasy.plid = ar.plid;''',cnx)

#dfWR = pd.read_sql_query('''SELECT fantasy.*,
                            

In [41]:
df.head(20)

,player,tm,pos,age,G,gs,plid,tmid,cmp_g,passatt_g,...,tm_passatt,passrate,tm_rushatt,avgdot,ybc_g,ydsbc_rec,yac_g,ydsac_rec,brktkl_g,drop_g
0,AJBrown,TEN,WR,24,13,13,AJBrown2021,TEN2021,0.000,0.154,...,31.471,90.0,32.412,11.6,47.769,9.9,19.077,3.9,0.154,0.615
1,AJBrown,TEN,WR,23,14,12,AJBrown2020,TEN2020,0.000,0.000,...,30.312,105.9,32.562,10.9,45.929,9.2,30.857,6.2,0.929,0.357
2,AJBrown,TEN,WR,22,16,11,AJBrown2019,TEN2019,0.000,0.000,...,28.000,108.3,27.812,13.2,36.625,11.3,29.062,8.9,0.500,0.125
3,AJDillon,GNB,RB,23,17,2,AJDillon2021,GNB2021,0.000,0.000,...,34.882,107.4,26.235,0.4,0.176,0.1,18.235,9.1,0.294,0.000
4,AJDillon,GNB,RB,22,11,0,AJDillon2020,GNB2020,0.000,0.000,...,32.875,121.5,27.688,2.5,0.455,2.5,1.455,8.0,0.091,0.000
5,AJGreen,ARI,WR,33,16,9,AJGreen2021,ARI2021,0.000,0.000,...,34.765,100.6,29.176,12.2,41.688,12.4,11.312,3.4,0.312,0.312
6,AJGreen,CIN,WR,32,16,14,AJGreen2020,CIN2020,0.000,0.000,...,36.312,85.7,25.688,13.7,27.312,9.3,5.375,1.8,0.125,0.188
7,AaronJones,GNB,RB,27,15,15,AaronJones2021,GNB2021,0.000,0.000,...,34.882,107.4,26.235,0.9,-1.400,-0.4,27.467,7.9,0.267,0.200
8,AaronJones,GNB,RB,26,14,14,AaronJones2020,GNB2020,0.000,0.000,...,32.875,121.5,27.688,1.2,-0.714,-0.2,26.071,7.8,0.429,0.286
9,AaronJones,GNB,RB,25,16,16,AaronJones2019,GNB2019,0.000,0.000,...,35.812,95.3,25.688,2.8,2.562,0.8,27.062,8.8,0.625,0.125


In [42]:
# Check column choices
df.columns

Index(['player', 'tm', 'pos', 'age', 'G', 'gs', 'plid', 'tmid', 'cmp_g',
       'passatt_g', 'passyds_g', 'passtd_g', 'int_g', 'rushatt_g', 'rushyds_g',
       'rushyds_att', 'rushtd_g', 'tgt_g', 'rec_g', 'recyds_g', 'yds_rec',
       'rectd_g', 'fmb_g', 'fl_g', 'tottd_g', '2PM_G', '2PP_G', 'fpts',
       'posrk', 'ovrank', 'yr', 'ppr_g', 'starter', 'next_yr_ppg',
       'next_yr_starter', 'tm_passatt', 'passrate', 'tm_rushatt', 'avgdot',
       'ybc_g', 'ydsbc_rec', 'yac_g', 'ydsac_rec', 'brktkl_g', 'drop_g'],
      dtype='object')

In [43]:
# Manipulate df to add rush and target share
df['rushshare'] = df['rushatt_g'] / df['tm_rushatt']
df['targetshare'] = df['tgt_g'] / df['tm_passatt']
df

,player,tm,pos,age,G,gs,plid,tmid,cmp_g,passatt_g,...,tm_rushatt,avgdot,ybc_g,ydsbc_rec,yac_g,ydsac_rec,brktkl_g,drop_g,rushshare,targetshare
0,AJBrown,TEN,WR,24,13,13,AJBrown2021,TEN2021,0.000,0.154,...,32.412,11.6,47.769,9.9,19.077,3.9,0.154,0.615,0.004751,0.256649
1,AJBrown,TEN,WR,23,14,12,AJBrown2020,TEN2020,0.000,0.000,...,32.562,10.9,45.929,9.2,30.857,6.2,0.929,0.357,0.000000,0.249769
2,AJBrown,TEN,WR,22,16,11,AJBrown2019,TEN2019,0.000,0.000,...,27.812,13.2,36.625,11.3,29.062,8.9,0.500,0.125,0.006760,0.187500
3,AJDillon,GNB,RB,23,17,2,AJDillon2021,GNB2021,0.000,0.000,...,26.235,0.4,0.176,0.1,18.235,9.1,0.294,0.000,0.419287,0.062382
4,AJDillon,GNB,RB,22,11,0,AJDillon2020,GNB2020,0.000,0.000,...,27.688,2.5,0.455,2.5,1.455,8.0,0.091,0.000,0.151040,0.005536
5,AJGreen,ARI,WR,33,16,9,AJGreen2021,ARI2021,0.000,0.000,...,29.176,12.2,41.688,12.4,11.312,3.4,0.312,0.312,0.000000,0.165396
6,AJGreen,CIN,WR,32,16,14,AJGreen2020,CIN2020,0.000,0.000,...,25.688,13.7,27.312,9.3,5.375,1.8,0.125,0.188,0.000000,0.179004
7,AaronJones,GNB,RB,27,15,15,AaronJones2021,GNB2021,0.000,0.000,...,26.235,0.9,-1.400,-0.4,27.467,7.9,0.267,0.200,0.434534,0.124219
8,AaronJones,GNB,RB,26,14,14,AaronJones2020,GNB2020,0.000,0.000,...,27.688,1.2,-0.714,-0.2,26.071,7.8,0.429,0.286,0.518528,0.136882
9,AaronJones,GNB,RB,25,16,16,AaronJones2019,GNB2019,0.000,0.000,...,25.688,2.8,2.562,0.8,27.062,8.8,0.625,0.125,0.574198,0.118675


In [44]:
# List out columns
df.columns

Index(['player', 'tm', 'pos', 'age', 'G', 'gs', 'plid', 'tmid', 'cmp_g',
       'passatt_g', 'passyds_g', 'passtd_g', 'int_g', 'rushatt_g', 'rushyds_g',
       'rushyds_att', 'rushtd_g', 'tgt_g', 'rec_g', 'recyds_g', 'yds_rec',
       'rectd_g', 'fmb_g', 'fl_g', 'tottd_g', '2PM_G', '2PP_G', 'fpts',
       'posrk', 'ovrank', 'yr', 'ppr_g', 'starter', 'next_yr_ppg',
       'next_yr_starter', 'tm_passatt', 'passrate', 'tm_rushatt', 'avgdot',
       'ybc_g', 'ydsbc_rec', 'yac_g', 'ydsac_rec', 'brktkl_g', 'drop_g',
       'rushshare', 'targetshare'],
      dtype='object')

In [45]:
# Select columns for each position (WR + TE joined in recvar)
qbvar = ['age', 'G', 'gs', 'passatt_g','passyds_g','passtd_g','rushatt_g','rushyds_g',
         'rushyds_att','ppr_g','starter','next_yr_starter']
rbvar = ['age', 'G', 'gs', 'rushatt_g','rushyds_g','rushyds_att','tgt_g','rec_g',
         'recyds_g','yds_rec','ppr_g','rushshare','targetshare']
# Added the following columns 'avgdot','ybc_g', 'ydsbc_rec', 'yac_g', 'ydsac_rec'
recvar = ['age', 'G', 'gs', 'tgt_g','rec_g','recyds_g','yds_rec','passrate', 'avgdot', 'ybc_g','ydsbc_rec', 'yac_g', 'ydsac_rec', 'brktkl_g', 'drop_g', 'targetshare' ]

In [46]:
# Separate into training, testing, using, and actual data by Yr (Fantasy) or Year (other dataframes)
train = df[df['yr'] <= 2019]
test = df[(df['yr'] >= 2019) & (df['yr'] <= 2020)]
use = df[df['yr'] == 2020]
use = use[use['next_yr_starter'].notna()]
use = use.astype({"next_yr_starter": int})
actuals = df[df['yr'] == 2021]

# Changed train from 2012 - 2019
# test 2018 and 2019
#use 2019
#actual 2020

In [47]:
# Name model and put in position
QBmodel = model(train, "QB", qbvar)
RBmodel = model(train, "RB", rbvar)
WRmodel = model(train, "WR", recvar)
TEmodel = model(train, "TE", recvar)
testModel(QBmodel, test, train, "QB", qbvar)
testModel(RBmodel, test, train, "RB", rbvar)
testModel(WRmodel, test, train, "WR", recvar)
testModel(TEmodel, test, train, "TE", recvar)

The accuracy of the QB model is 1.0
The QB model has an average error of 5.908 PPG and an average absolute error of 8.931 PPG
The QB model has a median error of 0.0 PPG and a median absolute error of 4.986 PPG


The accuracy of the RB model is 0.6401896176412283
The RB model has an average error of -0.151 PPG and an average absolute error of 2.778 PPG
The RB model has a median error of 0.153 PPG and a median absolute error of 2.147 PPG


The accuracy of the WR model is 0.6547541231394518
The WR model has an average error of 0.389 PPG and an average absolute error of 2.714 PPG
The WR model has a median error of 0.673 PPG and a median absolute error of 2.104 PPG


The accuracy of the TE model is 0.6969119733372908
The TE model has an average error of 0.079 PPG and an average absolute error of 1.952 PPG
The TE model has a median error of 0.516 PPG and a median absolute error of 1.601 PPG




In [12]:
# Use model to make predictions and check predictions
QBStats = useModel(QBmodel, use, actuals, 'QB', qbvar)
RBStats = useModel(RBmodel, use, actuals,'RB', rbvar)
WRStats = useModel(WRmodel, use, actuals, 'WR', recvar)
TEStats = useModel(TEmodel, use, actuals, 'TE', recvar)

The QB model has an average error of 6.191 PPG and an average absolute error of 11.177 PPG
The QB model has a median error of 6.191 PPG and a median absolute error of 11.177 PPG
The RB model has an average error of -0.427 PPG and an average absolute error of 2.805 PPG
The RB model has a median error of -0.048 PPG and a median absolute error of 2.175 PPG
The WR model has an average error of 0.724 PPG and an average absolute error of 2.827 PPG
The WR model has a median error of 1.016 PPG and a median absolute error of 2.217 PPG
The TE model has an average error of 0.19 PPG and an average absolute error of 2.059 PPG
The TE model has a median error of 0.654 PPG and a median absolute error of 1.709 PPG


In [13]:
pd.set_option('display.max_rows', None)
QBStats

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
0,AaronRodgers,QB,38.199,20.831,17.368221,649.383,17.368221
1,TaysomHill,QB,5.156,10.142,-4.986059,87.652,4.986059


In [14]:
QBStats.sort_values(by=['AbsDifference'])

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
1,TaysomHill,QB,5.156,10.142,-4.986059,87.652,4.986059
0,AaronRodgers,QB,38.199,20.831,17.368221,649.383,17.368221


In [15]:
RBStats

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
0,ChristianMcCaffrey,RB,21.867,18.214,3.653058,371.739,3.653058
1,AlvinKamara,RB,20.720,18.054,2.665630,352.240,2.665630
2,DalvinCook,RB,19.296,15.869,3.426898,328.032,3.426898
3,JamesRobinson,RB,16.949,12.421,4.528033,288.133,4.528033
4,DerrickHenry,RB,16.503,24.162,-7.658964,280.551,7.658964
5,AaronJones,RB,15.822,15.267,0.555034,268.974,0.555034
6,DavidMontgomery,RB,15.758,15.000,0.757659,267.886,0.757659
7,JonathanTaylor,RB,15.045,21.947,-6.901959,255.765,6.901959
8,NickChubb,RB,14.925,15.379,-0.453935,253.725,0.453935
9,MilesSanders,RB,14.426,9.767,4.659051,245.242,4.659051


In [ ]:
RBStats.sort_values(by=['AbsDifference'])

In [39]:
WRStats

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
0,DavanteAdams,WR,20.719,21.519,-0.799723,352.223,0.799723
1,StefonDiggs,WR,19.145,16.794,2.350797,325.465,2.350797
2,JustinJefferson,WR,18.272,19.435,-1.163412,310.624,1.163412
3,DeAndreHopkins,WR,16.650,14.720,1.930165,283.050,1.930165
4,AJBrown,WR,16.646,13.915,2.731185,282.982,2.731185
5,CalvinRidley,WR,16.449,14.220,2.228977,279.633,2.228977
6,DKMetcalf,WR,16.431,14.371,2.059544,279.327,2.059544
7,CoreyDavis,WR,15.889,11.689,4.200061,270.113,4.200061
8,TylerLockett,WR,15.834,15.088,0.746483,269.178,0.746483
9,WillFuller,WR,15.596,4.300,11.296291,265.132,11.296291


In [19]:
WRStats.sort_values(by=['AbsDifference'])

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
74,MecoleHardman,WR,8.364,8.288,0.076235,142.188,0.076235
140,CodyHollister,WR,2.328,2.433,-0.105346,39.576,0.105346
152,JakeKumerow,WR,0.532,0.320,0.212477,9.044,0.212477
47,MichaelGallup,WR,10.405,10.167,0.237625,176.885,0.237625
114,RayRayMcCloud,WR,3.796,4.138,-0.341784,64.532,0.341784
149,TommyleeLewis,WR,1.239,0.850,0.388741,21.063,0.388741
23,AmariCooper,WR,13.095,13.500,-0.404931,222.615,0.404931
21,MarquiseBrown,WR,13.735,14.144,-0.408644,233.495,0.408644
148,DedeWestbrook,WR,1.562,1.120,0.442170,26.554,0.442170
105,TylerJohnson,WR,4.690,4.235,0.455363,79.730,0.455363


In [ ]:
TEStats

In [ ]:
TEStats.sort_values(by=['AbsDifference'])